### On top of the Embeddedings gained from Bertweet, I added simple Neural network to 

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [13]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from sklearn.model_selection import train_test_split

from torch.utils.data import TensorDataset, DataLoader
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

In [27]:
if torch.cuda.is_available():       
    device = torch.device("cuda")
    print(f'There are {torch.cuda.device_count()} GPU(s) available.')
    print('Device name:', torch.cuda.get_device_name(0))

else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

No GPU available, using the CPU instead.


In [6]:
pheme = pd.read_csv('./data/_PHEME_text_AVGw2v.csv').drop(['token'],axis=1)
pheme_y = pd.read_csv('./data/_PHEME_target.csv').target
pheme_event = pd.read_csv('./data/_PHEME_text.csv').Event

ext = pd.read_csv('./data/_PHEMEext_text_AVGw2v.csv').drop(['token'],axis=1)
ext_y = pd.read_csv('./data/_PHEMEext_text.csv').target
ext_event = pd.read_csv('./data/_PHEMEext_text.csv').Event

rhi = pd.read_csv('./data/_RHI_text_AVGw2v.csv').drop(['token'],axis=1)
rhi_y = pd.read_csv('./data/_RHI_target.csv')

### Data with BERTweet Embedding consists of 797 Dimensions: which are 767 Embeddings and 30 additional features

### Other Dataset consists of 328 dimensions

## Print to Test

In [5]:
print("Shape of the Train and test data of Averaged Word2Vec: {}/{}".format(data.shape, test_data.shape))
# print("Shape of the Train and test data of Doc2vec: {}/{}".format(train_doc.shape, test_doc.shape))
# print("\nShape of the validation data of Avg: {}".format(valid_avg.shape))
# print("Shape of the validation data of Doc2vec: {}".format(valid_doc.shape))
# print("\nShape of the data w/ BERTweet: {}".format(df_bertweet.shape))
# print("\nShape of the validation data w/ BERTweet: {}".format(df_valid_bertweet.shape))

Shape of the Train and test data of Averaged Word2Vec: (5802, 200)/(5227, 200)


In [6]:
# Dropping Target values from the dataset
# train_y = train_avg.isRumor
# test_y = test_avg.isRumor
# valid_y = valid_avg.isRumor
# df_bertweet_y = df_bertweet.isRumor
# df_valid_bertweet_y = df_valid_bertweet.isRumor
# for dataset in [train_avg, test_avg, train_doc, test_doc, valid_avg, valid_doc, df_bertweet, df_valid_bertweet]:
#     dataset.drop(['isRumor'], axis=1, inplace=True)

# print(train.shape, test_y.shape, valid_y.shape, df_bertweet_y.shape, df_valid_bertweet_y.shape)
print(y.shape, test_y.shape)

(5802, 1) (5227, 1)


In [7]:
# # Base dataset are the baseline feature set to be inputted to the model
# # Here, 4 features are dropped for their lack of predictive power

# train_avg_base = train_avg.drop(['hasURL', 'hasUserURL', 'isNotOnlyText', 'char_count'],axis=1)
# test_avg_base = test_avg.drop(['hasURL', 'hasUserURL', 'isNotOnlyText', 'char_count'],axis=1)
# valid_avg_base = valid_avg.drop(['hasURL', 'hasUserURL', 'isNotOnlyText', 'char_count'],axis=1)
# train_doc_base = train_doc.drop(['hasURL', 'hasUserURL', 'isNotOnlyText', 'char_count'],axis=1)
# test_doc_base = test_doc.drop(['hasURL', 'hasUserURL', 'isNotOnlyText', 'char_count'],axis=1)
# valid_doc_base = valid_doc.drop(['hasURL', 'hasUserURL', 'isNotOnlyText', 'char_count'],axis=1)
# bertweet_base = df_bertweet.drop(['hasURL', 'hasUserURL', 'isNotOnlyText', 'char_count'],axis=1)
# bertweet_valid_base = df_valid_bertweet.drop(['hasURL', 'hasUserURL', 'isNotOnlyText', 'char_count'],axis=1)

# Torch

## Train-test-split을 이용한 기존 방법

In [ ]:
df
# # 위의 데이터들을 torchTensor로 변환한뒤 Unsqueeze한다.
# # 이후 TensorDataset를 생성한다. (X, y 값을 담은 텐서들을 인자로 넘겨줌)

# # tensor_x = torch.Tensor(np.array(data))
# # tensor_y = torch.Tensor(np.array(y))
# tensor_x = torch.tensor(data.values)
# tensor_y = torch.tensor(y.values)

# tensor_x = tensor_x.unsqueeze(1)
# # tensor_y = tensor_y.unsqueeze(1)

# data_dataset = TensorDataset(tensor_x,tensor_y)

# 훈련 데이터 / 테스트 데이터를 torch.utils.data.random_split()를 통해서 나눠준다
input_len = len(task1_dataset)
test_ratio = 0.1
test_size = int(input_len * test_ratio)
train_size = input_len - test_size

print("Length of the Inputs are: ",input_len, train_size, test_size)

train_data, test_data = torch.utils.data.random_split(task1_dataset, (train_size, test_size))
print(len(train_data), len(test_data))

# 위에서 생성한 훈련/테스트 데이터를 각각 DataLoader를 호출해 데이터 로더를 생성한다.
# 참고로 이 코드에서는 task1_dataset -> tensor_x/y -> train_avg_base/train_y를 사용하고 있다.
task1_train_dataloader = DataLoader(train_data, batch_size=6, shuffle=True, num_workers=2)
task1_test_dataloader = DataLoader(test_data, batch_size=6, shuffle=True, num_workers=2)

In [38]:
# FC_net을 생성 -> 
class FC_net(nn.Module):
    def __init__(self):
        super(FC_net, self).__init__() # 1*20
        self.fc1 = nn.Linear(200, 130) # 420
        self.fc2 = nn.Linear(130, 60)
        self.fc3 = nn.Linear(60, 1)

        self.drop_2 = nn.Dropout(0.2)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = torch.sigmoid(self.fc3(x))
        return x
        
task1_model = FC_net()

## BERTCLF 노트북을 참고한 방법

In [74]:
train_inputs = torch.tensor(pheme.values).float()
val_inputs = torch.tensor(ext.values).float()

# Convert other data types to torch.Tensor
train_labels = torch.tensor(pheme_y.values)
val_labels = torch.tensor(ext_y.values)

# For fine-tuning BERT, the authors recommend a batch size of 16 or 32.
batch_size = 32

# Create the DataLoader for our training set
train_data = TensorDataset(train_inputs, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

# Create the DataLoader for our validation set
val_data = TensorDataset(val_inputs, val_labels)
val_sampler = SequentialSampler(val_data)
val_dataloader = DataLoader(val_data, sampler=val_sampler, batch_size=batch_size)

In [75]:
print("Length of the Inputs are:\nTrain\t%s\nVal\t%s" %(train_inputs.size(), val_inputs.size()))

Length of the Inputs are:
Train	torch.Size([5802, 200])
Val	torch.Size([485, 200])


In [128]:
import random

class AvgW2VNet(torch.nn.Module):
  def __init__(self, freeze_bert=True):
    """
    In the constructor we construct three nn.Linear instances that we will use
    in the forward pass.
    """
    super(AvgW2VNet, self).__init__()

    D_in, H, D_out = 200, 16, 2

    self.classifier = nn.Sequential(
        nn.Linear(D_in, H),
        nn.ReLU(),
        # nn.Dropout(0.2),
        nn.Linear(H, D_out),
        nn.Sigmoid()
    )
    self.drop_2 = nn.Dropout(0.2)

    if freeze_bert:
        for param in self.bert.parameters():
            param.requires_grad = False

  def forward(self, input):
    result = self.classifier(input)
    return result

In [121]:
# from torch.nn.optim import Adam

def initialize_model(epochs=200):
    """Initialize the Bert Classifier, the optimizer and the learning rate scheduler.
    """
    # Instantiate Bert Classifier
    model = AvgW2VNet(freeze_bert=False)

    # Tell PyTorch to run the model on GPU
    model.to(device)

    # Create the optimizer
    optimizer = optim.Adam(model.parameters())

    criterion = nn.BCELoss()
    # loss = nn.CrossEntropyLoss()

    # Total number of training steps
    total_steps = len(train_dataloader) * epochs

    # Set up the learning rate scheduler
    # scheduler = get_linear_schedule_with_warmup(optimizer,
    #                                             num_warmup_steps=0, # Default value
    #                                             num_training_steps=total_steps)
    return model, optimizer, criterion
    # return bert_classifier, optimizer, scheduler


In [120]:
import random
import time


def set_seed(seed_value=42):
    """Set seed for reproducibility.
    """
    random.seed(seed_value)
    np.random.seed(seed_value)
    torch.manual_seed(seed_value)
    torch.cuda.manual_seed_all(seed_value)

def train(model, train_dataloader, val_dataloader=None, epochs=4, evaluation=False):
    """Train the BertClassifier model.
    """
    # Start training loop
    print("Start training...\n")
    for epoch_i in range(epochs):
        # =======================================
        #               Training
        # =======================================
        # Print the header of the result table
        print(f"{'Epoch':^7} | {'Batch':^7} | {'Train Loss':^12} | {'Val Loss':^10} | {'Val Acc':^9} | {'Elapsed':^9}")
        print("-"*70)

        # Measure the elapsed time of each epoch
        t0_epoch, t0_batch = time.time(), time.time()

        # Reset tracking variables at the beginning of each epoch
        total_loss, batch_loss, batch_counts = 0, 0, 0

        # Put the model into the training mode
        model.train()

        # For each batch of training data...
        for step, batch in enumerate(train_dataloader):
            batch_counts +=1
            # Load batch to GPU
            b_input, b_labels = tuple(t.to(device) for t in batch)

            # Zero out any previously calculated gradients
            model.zero_grad()

            # Perform a forward pass. This will return logits.
            logits = model(b_input)

            # Compute loss and accumulate the loss values
            print(logits)
            print(b_labels)
            loss = criterion(logits, b_labels)
            batch_loss += loss.item()
            total_loss += loss.item()

            # Perform a backward pass to calculate gradients
            loss.backward()

            # Clip the norm of the gradients to 1.0 to prevent "exploding gradients"
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

            # Update parameters and the learning rate
            optimizer.step()
            scheduler.step()

            # Print the loss values and time elapsed for every 20 batches
            if (step % 20 == 0 and step != 0) or (step == len(train_dataloader) - 1):
                # Calculate time elapsed for 20 batches
                time_elapsed = time.time() - t0_batch

                # Print training results
                print(f"{epoch_i + 1:^7} | {step:^7} | {batch_loss / batch_counts:^12.6f} | {'-':^10} | {'-':^9} | {time_elapsed:^9.2f}")

                # Reset batch tracking variables
                batch_loss, batch_counts = 0, 0
                t0_batch = time.time()

        # Calculate the average loss over the entire training data
        avg_train_loss = total_loss / len(train_dataloader)

        print("-"*70)
        # =======================================
        #               Evaluation
        # =======================================
        if evaluation == True:
            # After the completion of each training epoch, measure the model's performance
            # on our validation set.
            val_loss, val_accuracy = evaluate(model, val_dataloader)

            # Print performance over the entire training data
            time_elapsed = time.time() - t0_epoch
            
            print(f"{epoch_i + 1:^7} | {'-':^7} | {avg_train_loss:^12.6f} | {val_loss:^10.6f} | {val_accuracy:^9.2f} | {time_elapsed:^9.2f}")
            print("-"*70)
        print("\n")
    
    print("Training complete!")

def evaluate(model, val_dataloader):
    """After the completion of each training epoch, measure the model's performance
    on our validation set.
    """
    # Put the model into the evaluation mode. The dropout layers are disabled during
    # the test time.
    model.eval()

    # Tracking variables
    val_accuracy = []
    val_loss = []

    # For each batch in our validation set...
    for batch in val_dataloader:
        # Load batch to GPU
        b_input, b_labels = tuple(t.to(device) for t in batch)

        # Compute logits
        with torch.no_grad():
            logits = model(b_input)

        # Compute loss
        loss = loss_fn(logits, b_labels)
        val_loss.append(loss.item())

        # Get the predictions
        preds = torch.argmax(logits, dim=1).flatten()

        # Calculate the accuracy rate
        accuracy = (preds == b_labels).cpu().numpy().mean() * 100
        val_accuracy.append(accuracy)

    # Compute the average accuracy and loss over the validation set.
    val_loss = np.mean(val_loss)
    val_accuracy = np.mean(val_accuracy)

    return val_loss, val_accuracy

In [126]:
set_seed(42)    # Set seed for reproducibility
model, optimizer, criterion = initialize_model(epochs=100)
# criterion = nn.BCELoss()
criterion = nn.BCEWithLogitsLoss()
# criterion = nn.CrossEntropyLoss()


In [127]:
train(model, train_dataloader, val_dataloader, epochs=100, evaluation=True)

Start training...

 Epoch  |  Batch  |  Train Loss  |  Val Loss  |  Val Acc  |  Elapsed 
----------------------------------------------------------------------
tensor([[0.5271, 0.4975],
        [0.5297, 0.4909],
        [0.5199, 0.5000],
        [0.5374, 0.4968],
        [0.5359, 0.4928],
        [0.5383, 0.5051],
        [0.5366, 0.4905],
        [0.5394, 0.4924],
        [0.5347, 0.4994],
        [0.5289, 0.4955],
        [0.5479, 0.4989],
        [0.5446, 0.4936],
        [0.5286, 0.4958],
        [0.5339, 0.4895],
        [0.5302, 0.5000],
        [0.5282, 0.4951],
        [0.5387, 0.4988],
        [0.5430, 0.5038],
        [0.5220, 0.5002],
        [0.5346, 0.4874],
        [0.5353, 0.5009],
        [0.5354, 0.4953],
        [0.5405, 0.5069],
        [0.5311, 0.4861],
        [0.5293, 0.4981],
        [0.5374, 0.5060],
        [0.5326, 0.4970],
        [0.5320, 0.4984],
        [0.5370, 0.5031],
        [0.5341, 0.4862],
        [0.5260, 0.4958],
        [0.5213, 0.4915]], grad_fn

ValueError: Target size (torch.Size([32])) must be the same as input size (torch.Size([32, 2]))

In [ ]:

# N is batch size; D_in is input dimension;
# H is hidden dimension; D_out is output dimension.
N, D_in, H, D_out = 64, 300, 8, 2

# Create random Tensors to hold inputs and outputs.
x = torch.randn(N, D_in)
y = torch.randn(N, D_out)

# Construct our model by instantiating the class defined above
model = AvgW2VNet(D_in, H, D_out)

# Construct our loss function and an Optimizer. Training this strange model with
# vanilla stochastic gradient descent is tough, so we use momentum
criterion = torch.nn.MSELoss(reduction='sum')
optimizer = torch.optim.SGD(model.parameters(), lr=1e-4, momentum=0.9)

for t in range(500):
  # Forward pass: Compute predicted y by passing x to the model
  y_pred = model(x)

  # Compute and print loss
  loss = criterion(y_pred, y)
  print(t, loss.item())

  # Zero gradients, perform a backward pass, and update the weights.
  optimizer.zero_grad()
  loss.backward()
  optimizer.step()

## adf

In [39]:
print(task1_model.parameters)

<bound method Module.parameters of FC_net(
  (fc1): Linear(in_features=792, out_features=130, bias=True)
  (fc2): Linear(in_features=130, out_features=60, bias=True)
  (fc3): Linear(in_features=60, out_features=1, bias=True)
  (drop_2): Dropout(p=0.2, inplace=False)
)>


In [40]:
import torch.optim as optim

criterion = nn.BCELoss()
# optimizer = optim.SGD(task1_model.parameters(), lr=0.001, momentum=0.9)
optimizer = optim.Adam(task1_model.parameters(), lr=0.01)

In [41]:
train_loss = []
train_accuracy = []

prev_loss = 10
PATH = "./state_dict_BERT_fc.pt"
best_acc = 10.0
num_epochs = 10

val_corrects_list = []
val_loss_list = []

In [42]:
for epoch in range(num_epochs):  # loop over the dataset multiple times
    print('Epoch {}/{}'.format(epoch, num_epochs - 1))
    print('-' * 10)

    running_loss = 0.0
    running_corrects = 0
    task1_model.train()  # Set model to training mode
    for i, data in enumerate(task1_train_dataloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data
        # inputs, labels = inputs.float(), labels.long()

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = task1_model(inputs)

        labels = labels.unsqueeze(1).float()
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()    

        running_loss += loss.item()
        running_corrects += torch.sum(outputs == labels.data)
        # print(running_corrects)

    epoch_loss = running_loss / train_size
    epoch_acc = running_corrects.double() / train_size
    train_loss.append(epoch_loss)
    train_accuracy.append(epoch_acc)

    print('Train) Loss: {:.4f} Acc: {:.4f}'.format(epoch_loss, epoch_acc))

    # if epoch_loss < best_acc:
    #     # print("prev_loss: {:.5f}".format(prev_loss))
    #     # print("loss: {:.5f}".format(loss))
    #     print("Saving the best model w/ loss {:.4f}".format(epoch_loss))
    #     torch.save(task1_model.state_dict(),PATH)
    #     best_acc = epoch_loss

Epoch 0/9
----------
Train) Loss: 0.3190 Acc: 0.0042
Epoch 1/9
----------
Train) Loss: 5.4705 Acc: 0.6325
Epoch 2/9
----------
Train) Loss: 5.7149 Acc: 0.6568
Epoch 3/9
----------
Train) Loss: 5.7213 Acc: 0.6568
Epoch 4/9
----------
Train) Loss: 5.7149 Acc: 0.6568
Epoch 5/9
----------
Train) Loss: 5.7213 Acc: 0.6568
Epoch 6/9
----------
Train) Loss: 5.7149 Acc: 0.6568
Epoch 7/9
----------
Train) Loss: 5.7149 Acc: 0.6568
Epoch 8/9
----------
Train) Loss: 5.7213 Acc: 0.6568
Epoch 9/9
----------
Train) Loss: 5.7213 Acc: 0.6568


In [43]:
task1_model.eval()
correct = 0
total = 0
test_loss = 0
outputs_list = []
y_list = []

with torch.no_grad():
    val_loss = 0

    for i, data in enumerate(task1_test_dataloader):
        x, y = data
        x, y = x.float(), y.long()
        outputs = task1_model(x)
        loss = criterion(outputs, y.unsqueeze(1).float())
        _, predicted = torch.max(outputs.data, 1)
        outputs_list.append(predicted[:])
        total += y.size(0)
        correct += (predicted == y).double().sum().item()
        val_loss += loss.item()
        y_list.append(y)

print('Accuracy of the test dataset is: %d %%' % (100 * correct / total))
print("Loss of validation set: {:.5f}".format((val_loss / test_size)))
acc = (100 * correct / total)

Accuracy of the test dataset is: 68 %
Loss of validation set: 5.20883
